# COGS 189 Final Project 

**Team Name**  
Argus

**Team Members**  
Xiaolong Huang	(xih002@ucsd.edu),  
Bohan Lei		(blei@ucsd.edu),  
Mingze Xu		(m6xu@ucsd.edu),  
Weijia Zeng		(wezeng@ucsd.edu)     

**Project Goal**
- To fully understand and implement the vanilla Common Spatial Pattern (CSP) algorithm for EEG pattern recognition.
- To experiment with different types of Common Spatial Pattern (CSP) and Linear Discriminant Analysis (LDA) algorithms in motor imagery signal pattern classification.
- To improve motor imagery signal pattern classification accuracy by different CSP and LDA hyperparameter configurations.

**Sources**
- Data source: https://github.com/bregydoc/bcidatasetIV2a
- Data explanation: https://www.bbci.de/competition/iv/desc_2a.pdf
- CSP paper: https://ieeexplore.ieee.org/document/4408441
- SACSP paper: https://cogsci.ucsd.edu/~desa/Winter_conference_on_bci_2022.pdf

## Section 1: Setup

In [2]:
import numpy as np
from scipy.linalg import eigh
from scipy.signal import butter, sosfiltfilt, sosfreqz
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.linear_model import LinearRegression as LR
import scipy.linalg

## Section 2: Data Import

In [3]:
n_subject = 9

data_dir = './data/'
data_prefix = 'A'
data_suffix = '.npz'
data_type = {'train':'T', 'test':'E'}
data_path = data_dir + data_prefix + '{subject:02d}{type_:s}' + data_suffix

In [4]:
raw_train_subject = []
raw_test_subject = []

for subject_num in range(1, n_subject+1):
    train_path = data_path.format(subject=subject_num, type_=data_type['train'])
    test_path  = data_path.format(subject=subject_num, type_=data_type['test'])
    raw_train_subject.append(np.load(train_path))
    raw_test_subject.append(np.load(test_path))

In [5]:
class_code = {'left':769, 'right':770, 'foot':771, 'tongue':772}

In [6]:
# globally
fs = 250                                    # sampling frequency

# within in each epoch
cue_start = 2.                              # cue starts at
cue_end = 3.25                              # cue ends at
smr_start = 3.                              # smr starts after
smr_end = 6.                                # smr ends after
cue_start_offset = round(cue_start * fs)    # the first cue sample is at 
cue_end_offset   = round(cue_end * fs)      # the last cue sample is at
smr_start_offset = round(smr_start * fs)    # the first smr sample is after
smr_end_offset   = round(smr_end * fs)      # the last smr sample is after

In [7]:
train_subject = []
for raw_data in raw_train_subject: 
    signal = raw_data['s'].T
    cue_class = raw_data['etyp']
    cue_position = raw_data['epos']
    
    X_train = {}
    
    for name, code in class_code.items():
        epoch_start = cue_position[cue_class==code] - cue_start_offset
        
        epochs = []
        for i in range(len(epoch_start)):
            epochs.append(signal[:, epoch_start[i]+smr_start_offset:epoch_start[i]+smr_end_offset])
        epochs = np.stack(epochs)
        
        X_train[name] = epochs
    
    train_subject.append(X_train)

In [9]:
test_subject = []
for raw_data in raw_test_subject: 
    signal = raw_data['s'].T
    cue_class = raw_data['etyp']
    cue_position = raw_data['epos']
    
    X_test = {}
    
    name = 'unknown'
    code = 783
    
    epoch_start = cue_position[cue_class==code] - cue_start_offset

    epochs = []
    for i in range(len(epoch_start)):
        epochs.append(signal[:, epoch_start[i]+smr_start_offset:epoch_start[i]+smr_end_offset])
    epochs = np.stack(epochs)

    X_test[name] = epochs
    
    test_subject.append(X_test)

In [12]:
test_subject[0]['unknown'].shape

(288, 25, 750)

## Section 3: Signal Processing

## Section 4: Common Spatial Pattern

In [ ]:
def compute_covariance_matrix(X):
    return np.sum(X @ X.transpose((0,2,1)), axis=0) / X.shape[0]

In [ ]:
def compute_spatial_filters(S1, S2):
    return scipy.linalg.eigh(S1, S1+S2)

## Section 5: Linear Discriminant Analysis

In [ ]:
# process data for training filter 


In [2]:
def Process_w(w, j):
    processed_w = np.delete(w, list(range(int(j/2), int(25-j/2))) , axis = 1)
                            
    return processed_w

In [3]:
def CSPFilter(data, w, label, j):
    w = np.array(w)
    label = label.reshape()
    data = np.array(data)
    filter = LDA()
    processed_data = []
    for k in range(data.shape[0]):
        processed_data_temp = [1]
        for i in range(0,j):
            temp_data = np.log(w[:,i].T @ data[k,:,:] @ data[k,:,:].T @ w[:,i])
            processed_data_temp.append(temp_data)
        processed_data.append(np.array(processed_data_temp))
    processed_data = np.vstack(processed_data)
    filter.fit(processed_data,label)
    return filter